# Urban Water Model - Cell Flow Analysis

This notebook provides interactive cell selection and flow visualization tools for analyzing water flows at the individual cell level.

In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import geopandas as gpd
from dynaconf import Dynaconf

from ipywidgets import Button, Output
import ipywidgets as widgets
import plotly.graph_objects as go

import holoviews as hv
from bokeh.io import output_notebook
hv.extension('bokeh')
output_notebook()

import plotly.graph_objects as go
from typing import Dict, List
import json
from IPython.display import display, HTML
import ipywidgets as widgets

from duwcm.read_data import read_data
from duwcm.forcing import read_forcing
from duwcm.water_model import UrbanWaterModel
from duwcm.water_balance import run_water_balance
from duwcm.utils import load_config
from duwcm.functions import select_cells
from duwcm.postprocess import extract_local_results
from duwcm.scenario_manager import ScenarioManager, Scenario, run_scenario
from duwcm.diagnostics import DiagnosticTracker

from duwcm.viz import (
    interactive_cell_selection,
    create_flows,
    create_cell_flows,
    create_map_base
)

## Configuration

Set up the configuration and load required data.

In [ ]:
config = load_config('.', 'default', 'config.yaml')

print(f"Input directory: {config.input_directory}")
print(f"Output directory: {config.output.directory}")
print(f"Simulation period: {config.simulation.start_year} - {config.simulation.end_year}")

geo_file = Path(config.input_directory) / Path(config.files.geo)
background_file = Path(config.geodata_directory) / config.files.background_shapefile

_, _, _, _, _, flow_paths = read_data(config)

# Initialize diagnostic tracker
tracker = DiagnosticTracker()

# Call the interactive selection function
selected_cells = interactive_cell_selection(config, geo_file, background_file, flow_paths)

## Scenarios
Configure the scenarios

In [ ]:
# Scenarios
scenarios = ['default', 'dry', 'wet', 'water_saving', 'high_consumption']
scenario_results: Dict[str, Dict] = {}

# Create parameter sliders for each scenario
scenario_params = {
    'dry': {
        'precipitation_factor': widgets.FloatSlider(value=0.5, min=0.0, max=2.0, step=0.1, description='Precip. Factor'),
        'indoor_water_factor': widgets.FloatSlider(value=1.0, min=0.0, max=2.0, step=0.1, description='Indoor Water')
    },
    'wet': {
        'precipitation_factor': widgets.FloatSlider(value=2.0, min=0.0, max=2.0, step=0.1, description='Precip. Factor'),
        'indoor_water_factor': widgets.FloatSlider(value=1.0, min=0.0, max=2.0, step=0.1, description='Indoor Water')
    },
    'water_saving': {
        'precipitation_factor': widgets.FloatSlider(value=1.0, min=0.0, max=2.0, step=0.1, description='Precip. Factor'),
        'indoor_water_factor': widgets.FloatSlider(value=0.5, min=0.0, max=2.0, step=0.1, description='Indoor Water')
    },
    'high_consumption': {
        'precipitation_factor': widgets.FloatSlider(value=1.0, min=0.0, max=2.0, step=0.1, description='Precip. Factor'), 
        'indoor_water_factor': widgets.FloatSlider(value=2.0, min=0.0, max=2.0, step=0.1, description='Indoor Water')
    }
}

# Create tabs for each scenario
scenario_tabs = widgets.Tab()
scenario_boxes = []

for scenario in scenarios[1:]:  # Skip default
    vbox = widgets.VBox([
        widgets.HTML(f"<h3>{scenario} Scenario Parameters</h3>"),
        scenario_params[scenario]['precipitation_factor'],
        scenario_params[scenario]['indoor_water_factor']
    ])
    scenario_boxes.append(vbox)

scenario_tabs.children = scenario_boxes
for i, scenario in enumerate(scenarios[1:]):
    scenario_tabs.set_title(i, scenario)

# Display parameter interface
display(scenario_tabs)

# Modify the scenarios loop to use widget values
def get_scenario_config(scenario: str) -> Dynaconf:
    """Get configuration with updated parameters from widgets"""
    scenario_config = load_config('.', 'default', 'scenarios.yaml')
    
    if scenario != 'default' and scenario in scenario_params:
        # Update parameters from widgets
        scenario_config.scenarios[scenario]['precipitation_factor'] = scenario_params[scenario]['precipitation_factor'].value
        scenario_config.scenarios[scenario]['indoor_water_factor'] = scenario_params[scenario]['indoor_water_factor'].value
    
    return scenario_config

## Run Simulation

Execute the urban water model simulation for selected cells.

In [ ]:
# Read and prepare model data
model_params, reuse_settings, demand_data, soil_data, et_data, flow_paths = read_data(config)
forcing_data = read_forcing(config)

# Filter selected cells if specified
if hasattr(config.grid, 'selected_cells'):
    model_params, flow_paths = select_cells(model_params, flow_paths, config.grid.selected_cells)

# Create trackers for each scenario
trackers = {
    scenario: DiagnosticTracker() 
    for scenario in scenarios
}

# Run with trackers
for scenario in scenarios:
    # Get scenario config
    scenario_config = get_scenario_config(scenario)
    scenario_manager = ScenarioManager.from_config(scenario_config)
    
    # Run scenario with tracker
    # Create modified parameters for each scenario run
    if scenario != 'default':
        # Get the scenario object
        scenario_obj = scenario_manager.scenarios[scenario]
        # Apply modifications
        modified_params = scenario_obj.modify_params(model_params.copy())
        modified_forcing = scenario_obj.modify_forcing(forcing_data.copy())
    else:
        modified_params = model_params
        modified_forcing = forcing_data
    
    # Run model with modified parameters
    model = UrbanWaterModel(
        params=modified_params,  # Use modified parameters
        path=flow_paths,
        soil_data=soil_data,
        et_data=et_data,
        demand_settings=demand_data,
        reuse_settings=reuse_settings,
        direction=config.grid.direction
    )
    
    results = run_water_balance(model, modified_forcing, tracker=trackers[scenario])  # Use modified forcing
    scenario_results[scenario] = results

## Cell Flow Analysis

Analyze internal and external flows for selected cells.

In [ ]:
# Create cell and scenario selectors
cell_selector = widgets.Dropdown(
    options=sorted(config.grid.selected_cells) if hasattr(config.grid, 'selected_cells') else sorted(model_params.keys()),
    description='Cell ID:'
)

scenario_selector = widgets.Dropdown(
    options=scenarios,
    description='Scenario:'
)

# Create output area for plots
plot_output = widgets.Output()

def update_plots(cell_id, scenario_name):
    """Update both flow diagrams for selected cell and scenario"""
    with plot_output:
        plot_output.clear_output()
        # Get results for selected scenario
        results = scenario_results[scenario_name]
        tracker = trackers[scenario_name]
        
        internal_fig, external_fig = create_cell_flows(cell_id, tracker)
        internal_fig.update_layout(title=f"{scenario_name} - Cell {cell_id} Internal Flows")
        external_fig.update_layout(title=f"{scenario_name} - Cell {cell_id} External Flows")
        
        internal_fig.show()
        external_fig.show()

def on_cell_change(change):
    update_plots(change.new, scenario_selector.value)

def on_scenario_change(change):
    update_plots(cell_selector.value, change.new)

cell_selector.observe(on_cell_change, names='value')
scenario_selector.observe(on_scenario_change, names='value')

# Display widgets
display(widgets.HBox([cell_selector, scenario_selector]))
display(plot_output)

# Initial plot
if cell_selector.options:
    update_plots(cell_selector.value, scenario_selector.value)